In [1]:
# Identifying tweets by the stock they are talking about before we drop the tweets in the conversation.
# Ctrl+f "Wyatt" to see my comments on the code.
import warnings
from time import time
import re
from typing import List

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import explode, udf, size, col, concat_ws, to_date, to_timestamp, date_trunc, count, array_contains, month, dayofmonth, hour, explode_outer
from pyspark.sql.types import StringType, ArrayType, Row

stocks = ["AAPL", "MSFT", "AMZN", "TSLA", "GOOG", "GOOGL", "META", "NVDA", "PEP", "COST", "AVGO", "CSCO", "TMUS",
          "ADBE", "TXN", "CMCSA", "AMGN", "QCOM", "NFLX", "HON", "INTU", "INTC", "SBUX", "PYPL", "ADP", "AMD",
          "GILD", "MDLZ", "REGN", "ISRG", "VRTX", "ADI", "BKNG", "AMAT", "FISV", "CSX", "MU", "ATVI", "KDP", "CHTR",
          "MAR", "MRNA", "PANW", "ORLY", "ABNB", "MNST", "LRCX", "KHC", "SNPS", "AEP", "ADSK", "CDNS", "MELI",
          "CTAS", "FTNT", "PAYX", "KLAC", "BIIB", "DXCM", "NXPI", "EXC", "ASML", "LULU", "EA", "XEL", "MCHP",
          "CRWD", "MRVL", "AZN", "ILMN", "PCAR", "DLTR", "CTSH", "WDAY", "ROST", "ODFL", "WBA", "CEG", "IDXX",
          "TEAM", "VRSK", "FAST", "CPRT", "PDD", "SGEN", "SIRI", "DDOG", "LCID", "ZS", "JD", "EBAY", "VRSN", "ZM",
          "ANSS", "BIDU", "ALGN", "SWKS", "MTCH", "SPLK", "NTES", "DOCU", "OKTA"]

cashtag_extract = "\$(?!((AAPL|MSFT|AMZN|TSLA|GOOG|GOOGL|META|NVDA|PEP|COST|AVGO|CSCO|TMUS|ADBE|TXN|CMCSA|AMGN|QCOM|NFLX|HON|INTU|INTC|SBUX|PYPL|ADP|AMD|GILD|MDLZ|REGN|ISRG|VRTX|ADI|BKNG|AMAT|FISV|CSX|MU|ATVI|KDP|CHTR|MAR|MRNA|PANW|ORLY|ABNB|MNST|LRCX|KHC|SNPS|AEP|ADSK|CDNS|MELI|CTAS|FTNT|PAYX|KLAC|BIIB|DXCM|NXPI|EXC|ASML|LULU|EA|XEL|MCHP|CRWD|MRVL|AZN|ILMN|PCAR|DLTR|CTSH|WDAY|ROST|ODFL|WBA|CEG|IDXX|TEAM|VRSK|FAST|CPRT|PDD|SGEN|SIRI|DDOG|LCID|ZS|JD|EBAY|VRSN|ZM|ANSS|BIDU|ALGN|SWKS|MTCH|SPLK|NTES|DOCU|OKTA)(?![a-z|A-Z])))\w+"
hashtag_extract = "\#(?!((AAPL|MSFT|AMZN|TSLA|GOOG|GOOGL|META|NVDA|PEP|COST|AVGO|CSCO|TMUS|ADBE|TXN|CMCSA|AMGN|QCOM|NFLX|HON|INTU|INTC|SBUX|PYPL|ADP|AMD|GILD|MDLZ|REGN|ISRG|VRTX|ADI|BKNG|AMAT|FISV|CSX|MU|ATVI|KDP|CHTR|MAR|MRNA|PANW|ORLY|ABNB|MNST|LRCX|KHC|SNPS|AEP|ADSK|CDNS|MELI|CTAS|FTNT|PAYX|KLAC|BIIB|DXCM|NXPI|EXC|ASML|LULU|EA|XEL|MCHP|CRWD|MRVL|AZN|ILMN|PCAR|DLTR|CTSH|WDAY|ROST|ODFL|WBA|CEG|IDXX|TEAM|VRSK|FAST|CPRT|PDD|SGEN|SIRI|DDOG|LCID|ZS|JD|EBAY|VRSN|ZM|ANSS|BIDU|ALGN|SWKS|MTCH|SPLK|NTES|DOCU|OKTA)(?![a-z|A-Z])))\w+"


def make_regex():
    regex = "((\$|\#|\＃)("
    for stock in stocks:
        regex = regex + stock + "|"
    regex = regex[:-1]
    regex = regex + ")(?![a-z|A-Z]))"
    return regex


ticker_matcher = make_regex()
print(ticker_matcher)
ticker_matcher = re.compile(ticker_matcher, re.IGNORECASE)


def write_db(save_df: DataFrame, table_name: str, mode="overwrite"):
    save_df.write.format("jdbc") \
        .mode(mode) \
        .option("url", "jdbc:mysql://127.0.0.1:3306/cs179g") \
        .option("driver", "com.mysql.cj.jdbc.Driver") \
        .option("dbtable", f"{table_name}") \
        .option("user", "group6") \
        .option("batchsize", "100000") \
        .option("password", "grp6").save()


def extract_cash_tags(col_value: str) -> list[str]:
    return list(map(lambda x: str(x).upper(), re.findall(r"$(\w+)", col_value))) if col_value else []


def extract_hash_tags(col_value: str) -> list[str]:
    return list(map(lambda x: str(x).upper(), re.findall(r"#(\w+)", col_value))) if col_value else []


def tweets_to_tickers(tweets, baseTweetText) -> list[str]:
    if tweets is None:
        return None
    if baseTweetText is None:
        return None
    tweets_text = baseTweetText
    # print(f"\nBase tweet's text:{tweets_text}")
    for item in tweets:
        tweets_text = tweets_text + "     " + item.text
    tweets_text = tweets_text + " "
    # print(f"\nConversation's text:{tweets_text}")

    stockList = ticker_matcher.findall(tweets_text)
    _stockList = []
    stockListEmpty = 1
    for item in stockList:
        _stockList.append(item[2].upper())
        stockListEmpty = 0
    stockList = [*set(_stockList)]
    stockList.sort()
    if stockListEmpty:
        print(f"\n\nStockList empty!:\n\n{tweets_text}\n\n")
    # print(stockList)
    return stockList


tweets_to_tickers_UDF = udf(
    lambda x, y: tweets_to_tickers(x, y), ArrayType(StringType()))





((\$|\#|\＃)(AAPL|MSFT|AMZN|TSLA|GOOG|GOOGL|META|NVDA|PEP|COST|AVGO|CSCO|TMUS|ADBE|TXN|CMCSA|AMGN|QCOM|NFLX|HON|INTU|INTC|SBUX|PYPL|ADP|AMD|GILD|MDLZ|REGN|ISRG|VRTX|ADI|BKNG|AMAT|FISV|CSX|MU|ATVI|KDP|CHTR|MAR|MRNA|PANW|ORLY|ABNB|MNST|LRCX|KHC|SNPS|AEP|ADSK|CDNS|MELI|CTAS|FTNT|PAYX|KLAC|BIIB|DXCM|NXPI|EXC|ASML|LULU|EA|XEL|MCHP|CRWD|MRVL|AZN|ILMN|PCAR|DLTR|CTSH|WDAY|ROST|ODFL|WBA|CEG|IDXX|TEAM|VRSK|FAST|CPRT|PDD|SGEN|SIRI|DDOG|LCID|ZS|JD|EBAY|VRSN|ZM|ANSS|BIDU|ALGN|SWKS|MTCH|SPLK|NTES|DOCU|OKTA)(?![a-z|A-Z]))


In [5]:
spark = SparkSession.builder.master(
    'local[4]').appName('workerTest').config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.30').getOrCreate()

#print("Reading data")
hashtag_df: DataFrame = spark.read.option("multiLine", "true").option("mode", "PERMISSIVE").json(
    "../Part2_test/hashtag_output/.*")

cashtag_df: DataFrame = spark.read.option("multiLine", "true").option("mode", "PERMISSIVE").json(
    "../Part2_test/cashtag_output/.*")

# print("Done")
#hashtag_df = hashtag_df.union(cashtag_df)
#print("Exploding tweets")
# Wyatt: We need to select col.includes.tweets as well to get the context of a tweet.
tweets_hashtag_data_df = hashtag_df.select(
    explode("tweets")).select("col.data.*", "col.includes.tweets")
tweets_cashtag_data_df = cashtag_df.select(
    explode("tweets")).select("col.data.*", "col.includes.tweets")
# print("Done")
count1 = tweets_hashtag_data_df.count()

tweets_hashtag_identified_df = tweets_hashtag_data_df.withColumn(
    "tickers", tweets_to_tickers_UDF(col("tweets"), col("text")))
tweets_cashtag_identified_df = tweets_cashtag_data_df.withColumn(
    "tickers", tweets_to_tickers_UDF(col("tweets"), col("text")))

count2 = tweets_hashtag_identified_df.count()

#print("Extracting cash and hashtags")
hashtags_df = tweets_hashtag_identified_df.select("author_id", "created_at", "public_metrics.*", "text", "tickers").withColumn(
    "tags", udf(extract_hash_tags, ArrayType(StringType()))("text"))

cashtags_df = tweets_cashtag_identified_df.select("author_id", "created_at", "public_metrics.*", "text", "tickers").withColumn(
    "tags", udf(extract_cash_tags, ArrayType(StringType()))("text"))

count3 = hashtags_df.count()

# Filtering out results from seperate cashtag and hashtag dataframes happens here
ebay_hashtags_df = hashtags_df.where(
    array_contains(col("tickers"), "EBAY"))
ebay_cashtags_df = cashtags_df.where(
    array_contains(col("tickers"), "EBAY"))

ebay_h_count = ebay_hashtags_df.count()
ebay_c_count = ebay_cashtags_df.count()
ebay_hashtags_df_f1 = ebay_hashtags_df.where(
    ~array_contains(col("tags"), "BOUTIQUE") & ~array_contains(col("tags"), "SEARCHNCOLLECT") & ~array_contains(col("tags"), "ALIEXPRESS"))
ebay_hashtags_df_f1_count = ebay_hashtags_df_f1.count()
ebay_hashtags_df_f2 = ebay_hashtags_df_f1.where(
    ~ebay_hashtags_df_f1.text.like("Check out%")
)
ebay_hashtags_df.show(10)
ebay_hashtags_df_f1.show(10)
ebay_hashtags_df_f2.show(10)
ebay_cashtags_df.show(10)

print(
    f"\nCount1-3 {count1}, {count2}, {count3}\nNumber of EBAY tweets with hashtags: {ebay_h_count}\nNumber of EBAY tweets with cashtags: {ebay_c_count}\nFilter 1: {ebay_hashtags_df_f1_count}\nFilter 2: {ebay_hashtags_df_f2.count()}\n")

# print("Done")
# print("Merging")
#  merge two dataframe
union_df = hashtags_df.union(cashtags_df).withColumn("created_at",
                                                     to_timestamp("created_at", "yyyy-MM-dd'T'HH:mm:ss.SSSX"))
# print("Done")
# print("Tags_df")
# tags_df = union_df.filter(size("tags") > 0).withColumn(
#     "tag", explode("tags"))
# # 1 top 5 stock with count #$ only
# tags_df.drop("tags").createOrReplaceTempView("tag")

# union_df.where(size('tickers') == 0).show(10)

# Now get popularity statistics of stocks
union_df = union_df.withColumn("month", month("created_at")).withColumn(
    "day", dayofmonth("created_at")).withColumn("hour", hour("created_at"))
union_df = union_df.select("month", "day", "hour", "like_count", "quote_count",
                           "reply_count", "retweet_count", "text", "tickers", "tags")
union_df.show(10)
# dataframe.withColumn("song", concat_ws(",", $"song"))
write_db(union_df.withColumn("tickers", concat_ws(",", union_df.tickers)
                             ).withColumn("tags", concat_ws(",", union_df.tags)), "union_df")
time_tickers_tags_df = union_df.select(
    "month", "day", "hour", "tickers", "tags")


22/11/22 04:50:41 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

22/11/22 04:50:49 WARN DataSource: All paths were ignored:
  file:/home/ubuntu/Part2_test/cashtag_output/.ipynb_checkpoints


[Stage 11:>                                                         (0 + 2) / 2]

StockList empty!:

RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz 


[Stage 17:>                                                         (0 + 2) / 2]

StockList empty!:

RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz 


[Stage 2

+-------------------+--------------------+----------+-----------+-----------+-------------+--------------------+-------+--------------------+
|          author_id|          created_at|like_count|quote_count|reply_count|retweet_count|                text|tickers|                tags|
+-------------------+--------------------+----------+-----------+-----------+-------------+--------------------+-------+--------------------+
|          321408110|2022-11-03T09:23:...|         0|          0|          0|            0|Check out  Handma...| [EBAY]|              [EBAY]|
|1467172731787563018|2022-11-03T09:23:...|         0|          0|          0|            0|SHOKZ OpenRun #wi...| [EBAY]|[WIRELESS, HEADPH...|
| 970484761687834624|2022-11-03T09:23:...|         0|          0|          0|            0|Blackdeer Outdoor...| [EBAY]|[EBAY, ALIEXPRESS...|
|           47434619|2022-11-03T09:24:...|         0|          0|          0|            6|RT @R1J2P: Take a...| [EBAY]|              [EBAY]|
|14989

+-------------------+--------------------+----------+-----------+-----------+-------------+--------------------+-------+--------------------+
|          author_id|          created_at|like_count|quote_count|reply_count|retweet_count|                text|tickers|                tags|
+-------------------+--------------------+----------+-----------+-----------+-------------+--------------------+-------+--------------------+
|          321408110|2022-11-03T09:23:...|         0|          0|          0|            0|Check out  Handma...| [EBAY]|              [EBAY]|
|1467172731787563018|2022-11-03T09:23:...|         0|          0|          0|            0|SHOKZ OpenRun #wi...| [EBAY]|[WIRELESS, HEADPH...|
|           47434619|2022-11-03T09:24:...|         0|          0|          0|            6|RT @R1J2P: Take a...| [EBAY]|              [EBAY]|
|1498979251537678339|2022-11-03T09:24:...|         0|          0|          0|            0|Check out Demon S...| [EBAY]|              [EBAY]|
|14999

...

+-------------------+--------------------+----------+-----------+-----------+-------------+--------------------+--------------------+----+
|          author_id|          created_at|like_count|quote_count|reply_count|retweet_count|                text|             tickers|tags|
+-------------------+--------------------+----------+-----------+-----------+-------------+--------------------+--------------------+----+
|1409159987289489412|2022-10-26T00:09:...|         0|          0|          0|            0|🔥Most active Tra...|[AAPL, AMD, AMZN,...|  []|
|1409159987289489412|2022-10-26T00:11:...|         0|          0|          0|            0|Check out our sub...|[AAPL, AMD, AMZN,...|  []|
|1409159987289489412|2022-10-26T00:16:...|         0|          0|          0|            0|Check out our dis...|[AAPL, AMD, EBAY,...|  []|
|1409159987289489412|2022-10-26T00:25:...|         0|          0|          0|            0|$TSLA $TAOP $JFK ...|[AAPL, AMD, AMZN,...|  []|
|1409159987289489412|2022-10

[Stage 24:>                                                         (0 + 2) / 2]

StockList empty!:

RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz 





Count1-3 47370, 47370, 47370
Number of EBAY tweets with hashtags: 20106
Number of EBAY tweets with cashtags: 527
Filter 1: 19148
Filter 2: 9888



+-----+---+----+----------+-----------+-----------+-------------+---------------------+-------+--------------------+
|month|day|hour|like_count|quote_count|reply_count|retweet_count|                 text|tickers|                tags|
+-----+---+----+----------+-----------+-----------+-------------+---------------------+-------+--------------------+
|   11|  3|   9|         0|          0|          0|            0| @GraphicArts77 #b...|  [KDP]|[BIBLIOPHILE, BOO...|
|   11|  3|   9|         0|          0|          0|           92| RT @Bloominje: Je...| [MRNA]|            [DOUDNA]|
|   11|  3|   9|         0|          0|          0|            0| @alexandriabaine ...|  [KDP]|[BIBLIOPHILE, BOO...|
|   11|  3|   9|         0|          0|          0|           60|RT @KBTEA1: 【11/2...|   [EA]|                  []|
|   11|  3|   9|         0|          0|          0|           60| RT @yoshimune_ptr...|   [EA]|                  []|
|   11|  3|   9|         0|          0|          0|            0|

[Stage 28:>                                                         (0 + 4) / 6]

StockList empty!:

RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz 




In [6]:
time_tickers_tags_df.filter(col('month') != 10).show()

+-----+---+----+-------+--------------------+
|month|day|hour|tickers|                tags|
+-----+---+----+-------+--------------------+
|   11|  3|   9|  [KDP]|[BIBLIOPHILE, BOO...|
|   11|  3|   9| [MRNA]|            [DOUDNA]|
|   11|  3|   9|  [KDP]|[BIBLIOPHILE, BOO...|
|   11|  3|   9|   [EA]|                  []|
|   11|  3|   9|   [EA]|                  []|
|   11|  3|   9| [META]|[METAVERSO, METAV...|
|   11|  3|   9| [META]|[META, POLYGON, T...|
|   11|  3|   9| [TSLA]|                  []|
|   11|  3|   9| [MRNA]|                  []|
|   11|  3|   9|  [KDP]|[BIBLIOPHILE, BOO...|
|   11|  3|   9| [META]|    [WHATSAPP, META]|
|   11|  3|   9| [MRNA]|              [MRNA]|
|   11|  3|   9|  [CSX]|                  []|
|   11|  3|   9|  [KDP]|[KDP, AMAZON, CHI...|
|   11|  3|   9| [COST]|[SUSTAINABLE, COS...|
|   11|  3|   9| [AAPL]|              [AAPL]|
|   11|  3|   9| [MRNA]|     [COVID19, MRNA]|
|   11|  3|   9| [EBAY]|              [EBAY]|
|   11|  3|   9| [META]|          

In [8]:
mdhr_tags = time_tickers_tags_df.withColumn("tickers", concat_ws(",", union_df.tickers)
                                             ).withColumn("tags", concat_ws(",", union_df.tags))

In [9]:
mdhr_tags.filter(col('month') != 10).show()

+-----+---+----+-------+--------------------+
|month|day|hour|tickers|                tags|
+-----+---+----+-------+--------------------+
|   11|  3|   9|    KDP|BIBLIOPHILE,BOOKL...|
|   11|  3|   9|   MRNA|              DOUDNA|
|   11|  3|   9|    KDP|BIBLIOPHILE,BOOKL...|
|   11|  3|   9|     EA|                    |
|   11|  3|   9|     EA|                    |
|   11|  3|   9|   META|METAVERSO,METAVER...|
|   11|  3|   9|   META|META,POLYGON,TOKE...|
|   11|  3|   9|   TSLA|                    |
|   11|  3|   9|   MRNA|                    |
|   11|  3|   9|    KDP|BIBLIOPHILE,BOOKL...|
|   11|  3|   9|   META|       WHATSAPP,META|
|   11|  3|   9|   MRNA|                MRNA|
|   11|  3|   9|    CSX|                    |
|   11|  3|   9|    KDP|KDP,AMAZON,CHILDR...|
|   11|  3|   9|   COST|SUSTAINABLE,COST,...|
|   11|  3|   9|   AAPL|                AAPL|
|   11|  3|   9|   MRNA|        COVID19,MRNA|
|   11|  3|   9|   EBAY|                EBAY|
|   11|  3|   9|   META|          

In [10]:
write_db(mdhr_tags, 'mdhr_tags')

[Stage 31:>                                                         (0 + 4) / 6]

StockList empty!:

RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     RT @aajtak: क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz     क्या Tech कंपनी भी आतंकवादी हो सकती है?

#BlackAndWhiteOnAajTak #Russia #Meta| @sudhirchaudhary https://t.co/zABL4GkTIz 


